<a href="https://colab.research.google.com/github/1004516/SE-ALES-Y-SISTEMAS/blob/main/Proyecto_Final_SyS/PROYECTO_FINAL_GIGA_SCIENCE_EEGMI_GCPDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
dggarciam94_giga_science_gcpds_path = kagglehub.dataset_download('dggarciam94/giga-science-gcpds')

print('Data source import complete.')


# Proyecto Final Señales y Sistemas 2025 -2

## Presentado por:
- Eduardo Yesid Aza Patiño
- Cristian David Chalaca Salas
- Gabriel Alejandro Gomez Mueses
## **Objetivo**: Implementar técnicas de representación en tiempo y frecuencia para el reconocimiento de señales de electroencefalografía (EEG) en tareas de imaginación motora (Motor Imagery)


![eegMI](https://figures.semanticscholar.org/288a54f091264377eccc99a19079c9387d66a78f/3-Figure2-1.png)

Las señales de EEG pueden ser ruidosas debido a diversas fuentes, incluidos artefactos fisiológicos e interferencias electromagnéticas. También pueden variar de persona a persona, lo que dificulta la extracción de características y la comprensión de las señales. Además, esta variabilidad, influenciada por factores genéticos y cognitivos, presenta desafíos para el desarrollo de soluciones independientes del sujeto.

**Base de datos**: GiGaScience Database [https://gigadb.org/dataset/100295](https://gigadb.org/dataset/100295)

Ver Sección 3.1 en [Multimodal Explainability Using Class Activation Maps and Canonical Correlation for MI-EEG Deep Learning Classification](https://www.mdpi.com/2076-3417/14/23/11208)

## Instalamos las librerias necesarias

## Ejercicio 1
Consultar para qué sirven las siguientes librerías

In [ ]:
#!pip install tensorflow==2.15.0
!pip install mne==1.6.0
!pip install braindecode===0.7
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases

**solucion**

**TensorFlow:**

TensorFlow es una de las bibliotecas más populares para aprendizaje profundo (deep learning). Sirve para:

* Construir y entrenar redes neuronales artificiales.
* Manejar grandes cantidades de datos para el aprendizaje automático.
* Implementar modelos de visión por computadora, procesamiento de lenguaje natural, reconocimiento de voz, etc.
  
**mne:**
  
MNE (Magnetoencephalography & Electroencephalography) es una librería especializada en el análisis de señales cerebrales como:

* EEG (Electroencefalografía): actividad eléctrica del cerebro.
* MEG (Magnetoencefalografía): campos magnéticos producidos por la actividad cerebral.
* ECG (Electrocardiografía): actividad eléctrica del corazón.

**Braindecode:**
Braindecode es una librería basada en PyTorch y MNE, especializada en aprendizaje profundo para el análisis de señales cerebrales. Se usa para:

* Clasificación de EEG en tareas de BCI (Interfaz Cerebro-Computadora).
* Extracción de características de señales cerebrales.
* Aplicaciones médicas y neurocientíficas.

**git+https://github.com/UN-GCPDS/python-gcpds.databases:**
  
Este paquete proviene del Grupo de Procesamiento de Señales de la Universidad de Navarra (GCPDS) y parece estar relacionado con bases de datos para señales biomédicas.

Al instalarlo desde GitHub, indica que el paquete está en desarrollo y proporciona acceso a:

* Bases de datos de EEG/MEG para experimentos en neurociencia.
* Herramientas para manejar y procesar señales biomédicas.

**Concluciones:**

* tensorflow: Para redes neuronales y aprendizaje profundo.
* mne: Para análisis de señales cerebrales EEG/MEG/ECG.
* braindecode: Para deep learning en neurociencia.
* gcpds.databases: Para acceder a bases de datos biomédicas.


## Importamos algunas librerias necesarias

In [ ]:
from scipy.signal import resample
from scipy.signal import freqz, filtfilt, resample
from scipy.signal import butter as bw
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
from gcpds.databases import GIGA_MI_ME
from sklearn.base import BaseEstimator, TransformerMixin

* resample: Cambia la frecuencia de muestreo de una señal.
* freqz: Muestra la respuesta en frecuencia de un filtro digital.
* filtfilt:Filtra una señal sin distorsión de fase (usa filtrado hacia adelante y hacia atrás).
* butter as bw:Diseña un filtro de Butterworth (ideal para señales EEG).

**Usos en EEG: Filtrado de señales, eliminación de ruido, re-muestreo de señales para análisis en distintas bandas de frecuencia.**
  
**pandas (Manejo de datos)**
Pandas es una de las librerías más utilizadas en manipulación y análisis de datos.

* Se usa para leer archivos CSV, estructurar datos en dataframes y realizar operaciones sobre ellos.
* Usos en EEG: Almacenar señales, etiquetas de clase (izquierda/derecha en BCI), tiempos de eventos.

**random (Generación de números aleatorios)**
* Módulo para generar números aleatorios en Python.
* Usos en EEG: Creación de conjuntos de entrenamiento/test, generación de ruido simulado.

**numpy (Cálculo numérico y manipulación de matrices)**
* NumPy es una librería optimizada para cálculos numéricos con matrices y vectores.
* Usos en EEG: Almacenamiento de señales, cálculos matemáticos en procesamiento de datos.

 **matplotlib.pyplot (Visualización de datos)**
* Matplotlib permite crear gráficos y visualizar datos.
* Usos en EEG: Graficar señales EEG, visualizar espectros de frecuencia.

**tensorflow (Desactivado en tu código)**
TensorFlow se usa para aprendizaje profundo, pero está comentado en tu código. Se usa en clasificación de señales cerebrales con redes neuronales.

*  Usos en EEG: Modelos de aprendizaje profundo para detección de patrones en señales.






## Funciones necesarias para el preprocesamiento leve de los datos

In [ ]:
def load_GIGA(db,
              sbj,
              eeg_ch_names,
              new_fs,
              fs,
              f_bank=None,
              vwt=None,
              run=None):

    index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

    #tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

    db.load_subject(sbj)
    if run == None:
        X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    else:
        X, y = db.get_run(run, classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    X = X[:, index_eeg_chs, :] #spatial rearrangement
    #X = np.squeeze(tf_repr.transform(X))
    #Resampling
    if new_fs == fs:
        pass#print('No resampling, since new sampling rate same.')
    else:
        print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)

    return X, y

def butterworth_digital_filter(X, N, Wn, btype, fs, axis=-1, padtype=None, padlen=0, method='pad', irlen=None):
  """
  Apply digital butterworth filter
  INPUT
  ------
  1. X: (D array)
    array with signals.
  2. N: (int+)
    The order of the filter.
  3. Wn: (float+ or 1D array)
    The critical frequency or frequencies. For lowpass and highpass filters, Wn is a scalar; for bandpass and bandstop filters, Wn is a length-2 vector.
    For a Butterworth filter, this is the point at which the gain drops to 1/sqrt(2) that of the passband (the “-3 dB point”).
    If fs is not specified, Wn units are normalized from 0 to 1, where 1 is the Nyquist frequency (Wn is thus in half cycles / sample and defined as 2*critical frequencies / fs). If fs is specified, Wn is in the same units as fs.
  4. btype: (str) {‘lowpass’, ‘highpass’, ‘bandpass’, ‘bandstop’}
    The type of filter
  5. fs: (float+)
    The sampling frequency of the digital system.
  6. axis: (int), Default=1.
    The axis of x to which the filter is applied.
  7. padtype: (str) or None, {'odd', 'even', 'constant'}
    This determines the type of extension to use for the padded signal to which the filter is applied. If padtype is None, no padding is used. The default is ‘odd’.
  8. padlen: (int+) or None, Default=0
    The number of elements by which to extend x at both ends of axis before applying the filter. This value must be less than x.shape[axis] - 1. padlen=0 implies no padding.
  9. method: (str), {'pad', 'gust'}
    Determines the method for handling the edges of the signal, either “pad” or “gust”. When method is “pad”, the signal is padded; the type of padding is determined by padtype
    and padlen, and irlen is ignored. When method is “gust”, Gustafsson’s method is used, and padtype and padlen are ignored.
  10. irlen: (int) or None, Default=nONE
    When method is “gust”, irlen specifies the length of the impulse response of the filter. If irlen is None, no part of the impulse response is ignored.
    For a long signal, specifying irlen can significantly improve the performance of the filter.
  OUTPUT
  ------
  X_fil: (D array)
    array with filtered signals.
  """
  b, a = bw(N, Wn, btype, analog=False, output='ba', fs=fs)
  return filtfilt(b, a, X, axis=axis, padtype=padtype, padlen=padlen, method=method, irlen=irlen)

class TimeFrequencyRpr(BaseEstimator, TransformerMixin):
  """
  Time frequency representation of EEG signals.

  Parameters
  ----------
    1. sfreq:  (float) Sampling frequency in Hz.
    2. f_bank: (2D array) Filter banks Frequencies. Default=None
    3. vwt:    (2D array) Interest time windows. Default=None
  Methods
  -------
    1. fit(X, y=None)
    2. transform(X, y=None)
  """
  def __init__(self, sfreq, f_bank=None, vwt=None):
    self.sfreq = sfreq
    self.f_bank = f_bank
    self.vwt = vwt
# ------------------------------------------------------------------------------

  def _validation_param(self):
    """
    Validate Time-Frequency characterization parameters.
    INPUT
    -----
      1. self
    ------
      2. None
    """
    if self.sfreq <= 0:
      raise ValueError('Non negative sampling frequency is accepted')


    if self.f_bank is None:
      self.flag_f_bank = False
    elif self.f_bank.ndim != 2:
      raise ValueError('Band frequencies have to be a 2D array')
    else:
      self.flag_f_bank = True

    if self.vwt is None:
      self.flag_vwt = False
    elif self.vwt.ndim != 2:
      raise ValueError('Time windows have to be a 2D array')
    else:
      self.flag_vwt = True

# ------------------------------------------------------------------------------
  def _filter_bank(self, X):
    """
    Filter bank Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_f: (4D array) set of filtered EEG signals, shape (trials, channels, time_samples, frequency_bands)
    """
    X_f = np.zeros((X.shape[0], X.shape[1], X.shape[2], self.f_bank.shape[0])) #epochs, Ch, Time, bands
    for f in np.arange(self.f_bank.shape[0]):
      X_f[:,:,:,f] = butterworth_digital_filter(X, N=5, Wn=self.f_bank[f], btype='bandpass', fs=self.sfreq)
    return X_f

# ------------------------------------------------------------------------------
  def _sliding_windows(self, X):
    """
    Sliding Windows Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_w: (4D array) shape (trials, channels, window_time_samples, number_of_windows)
    """
    window_lenght = int(self.sfreq*self.vwt[0,1] - self.sfreq*self.vwt[0,0])
    X_w = np.zeros((X.shape[0], X.shape[1], window_lenght, self.vwt.shape[0]))
    for w in np.arange(self.vwt.shape[0]):
        X_w[:,:,:,w] = X[:,:,int(self.sfreq*self.vwt[w,0]):int(self.sfreq*self.vwt[w,1])]
    return X_w

# ------------------------------------------------------------------------------
  def fit(self, X, y=None):
    """
    fit.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
      2. y: (1D array) target labels. Default=None
    OUTPUT
    ------
      1. None
    """
    pass

# ------------------------------------------------------------------------------
  def transform(self, X, y=None):
    """
    Time frequency representation of EEG signals.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, times)
    OUTPUT
    ------
      1. X_wf: (5D array) Time-frequency representation of EEG signals, shape (trials, channels, window_time_samples, number_of_windows, frequency_bands)
    """
    self._validation_param()     #Validate sfreq, f_freq, vwt

    #Avoid edge effects of digital filter, 1st:fbk, 2th:vwt
    if self.flag_f_bank:
        X_f = self._filter_bank(X)
    else:
        X_f = X[:,:,:,np.newaxis]

    if self.flag_vwt:
      X_wf = []
      for f in range(X_f.shape[3]):
        X_wf.append(self._sliding_windows(X_f[:,:,:,f]))
      X_wf = np.stack(X_wf, axis=-1)
    else:
      X_wf = X_f[:,:,:,np.newaxis,:]

    return X_wf

#plot eeg
def plot_eeg(X,tv,ax,channels,esp=2,title=None):
    # X in CH x Samples
    n_canales = X.shape[0]

    for ch in range(n_canales): # canales
            xx = X[ch]
            xx = xx - np.mean(xx)
            xx = xx/np.max(abs(xx))
            ax.plot(tv, xx +(ch * esp), label=channels[ch])  # Desplazamos cada canal para visualización
    ax.set_yticks(range(0, esp * n_canales, esp), channels)  # Etiquetas en el eje Y
    ax.set_xlabel("Tiempo [s]")
    ax.set_ylabel("Canales EEG [$\mu$V]")
    ax.set_title(title)
    ax.grid(True)
    ax.set_xlim([min(tv)-0.01,max(tv)+0.01])
    ax.set_ylim([-esp,n_canales*esp+0.01])







## Establecemos el protocolo de pruebas y la configuración del montaje EEG

Describir el protocolo de captura de datos y el montaje utilizado


![mi](https://www.mdpi.com/diagnostics/diagnostics-13-01122/article_deploy/html/images/diagnostics-13-01122-g001.png)
![montaje](https://www.mdpi.com/applsci/applsci-14-11208/article_deploy/html/images/applsci-14-11208-g001.png)

In [ ]:
channels = ['Fp1','Fpz','Fp2',
            'AF7','AF3','AFz','AF4','AF8',
            'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
            'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
            'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
            'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
            'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
            'PO7','PO3','POz','PO4','PO8',
            'O1','Oz','O2',
            'Iz']

areas = {
    'Frontal': ['Fpz', 'AFz', 'Fz', 'FCz'],
    'Frontal Right': ['Fp2','AF4','AF8','F2','F4','F6','F8',],
    'Central Right': ['FC2','FC4','FC6','FT8','C2','C4','C6','T8','CP2','CP4','CP6','TP8',],
    'Posterior Right': ['P2','P4','P6','P8','P10','PO4','PO8','O2',],
    #'Central': ['Cz'],
    'Posterior': ['CPz','Pz', 'Cz','POz','Oz','Iz',],
    'Posterior Left': ['P1','P3','P5','P7','P9','PO3','PO7','O1',],
    'Central Left': ['FC1','FC3','FC5','FT7','C1','C3','C5','T7','CP1','CP3','CP5','TP7',],
    'Frontal Left': ['Fp1','AF3','AF7','F1','F3','F5','F7',],
}

arcs = [
    #'hemispheres',
    'areas',
    'channels',
]

## Definimos la ruta y los argumentos para la carga de los datos de EEG

In [ ]:
db = GIGA_MI_ME('/kaggle/input/giga-science-gcpds/GIGA_MI_ME')
#ti = 0
#tf = 7
new_fs = 256.
load_args = dict(db = db,
                 eeg_ch_names = channels,
                 fs = db.metadata['sampling_rate'],
                 #f_bank = np.asarray([[4., 40.]]),
                 #vwt = np.asarray([[ti, tf]]), #2.5 - 5 MI
                 new_fs = new_fs)

## Cargamos los datos según el sujeto que se quiera

Si se quiere cargar los datos de todos los sujetos, aplicar un ciclo que itere la lista de sujetos y de esta forma se cargara uno por uno dependiendo lo que se desee realizar.

Por ejemplo:

for i in sbj:
    X, y = load_GIGA(sbj=sbj, **load_args)

In [ ]:
sbj = 5
X, y = load_GIGA(sbj=sbj, **load_args)

In [ ]:
print(f'X con {X.shape[0]} intentos; {X.shape[1]} canales; {X.shape[2]} muestras No. de segundos {X.shape[2]/new_fs}')

In [ ]:
X.shape

## Visualización de las señales de EEG en el tiempo

In [ ]:
#graficar canales promedio
trial = 0
ti = 0 # ti
tf = 7 # tf
tv = np.arange(ti,tf,1/new_fs)

#Señal cruda
fig,ax = plt.subplots(1,1,figsize=(8,8),sharex = True)
# Graficar cada canal en un subplot banda respectiva

plot_eeg(X[trial],tv,ax=ax,channels=channels,title='EEG original')
plt.show()

# Ejercicio 2

Discuta la gráfica anterior
**DISCUCION DE LA GRAFICA**

**Análisis de la Gráfica EEG en relación con los Ritmos Cerebrales**
La gráfica representa señales EEG multicanal en microvoltios (µV) vs. tiempo (s). Se observan múltiples trazas correspondientes a distintos electrodos ubicados en el cuero cabelludo, que registran la actividad eléctrica del cerebro.

A continuación, discutimos cómo se pueden relacionar los patrones observados con los ritmos cerebrales principales: delta, theta, alpha, beta y gamma.

*  La gráfica muestra múltiples canales EEG con oscilaciones características de la actividad cerebral.
 * Sin un análisis espectral (FFT o Wavelet), no podemos determinar con certeza qué ritmos predominan.
* Según la forma de las señales, podríamos suponer que hay actividad en los rangos de Alpha y Beta, y quizás Theta.
* Si el sujeto estaba relajado, esperaríamos Alpha; si estaba atento o realizando una tarea cognitiva, esperaríamos Beta.
* Para confirmar cada tipo de ritmo, debemos aplicar filtros de frecuencia específicos y realizar un análisis de espectro.


Nota: Discuta en qué consisten los ritmos cerebrales

![montaje](https://cdn.shopify.com/s/files/1/0348/7053/files/storage.googleapis.com-486681944373284_61cb9936-f6c2-493d-8402-3426d7f5a049_1024x1024.jpg?v=1689309340)



In [ ]:
# filtramos trials completos en ritmos cerebrales utilizando filtros IIR


f_bank = np.array([[0.5,4.],[4., 8.],[8.,13.],[13.,32.],[32.,100.]])
vwt = np.asarray([[ti, tf]]) #2.5 - 5 MI 0 - 7 trial completo
tf_repr = TimeFrequencyRpr(sfreq = new_fs, f_bank = f_bank)

Xrc = np.squeeze(tf_repr.transform(X))

Xrc.shape

# Ejercicio 3

Expliqué cómo se calcularon cada una de las 5 dimensiones del arreglo Xrc

**Solucion**

**1** primero se define la banda de frecuencias a filtrar:
Se establece un banco de filtros (f_bank) con 5 bandas de frecuencia correspondientes a los ritmos cerebrales principales:

f_bank = np.array([
    [0.5, 4.],    # Delta
    [4., 8.],     # Theta
    [8., 13.],    # Alpha
    [13., 32.],   # Beta
    [32., 100.]   # Gamma
])
estas bandas permiten separar la señal en diefrentes componentes.

**2**Definir la ventana de tiempo (vwt)
Se extrae un segmento específico de la señal EEG, por ejemplo, de 2.5 a 5 segundos para movimiento imaginado (MI) o de 0 a 7 segundos para un trial completo:

vwt = np.asarray([[ti, tf]])

**3**Aplicar la transformación de dominio tiempo-frecuencia
TimeFrequencyRpr (posiblemente basado en MNE o Braindecode) toma la señal EEG, filtra cada banda de frecuencia y genera una representación tiempo-frecuencia.

tf_repr = TimeFrequencyRpr(sfreq=new_fs, f_bank=f_bank)

**4**Transformar los datos EEG
La transformación filtra los datos y obtiene una nueva representación con las 5 dimensiones del arreglo Xrc:

Xrc = np.squeeze(tf_repr.transform(X))

**Relación con el Procesamiento de EEG**
Cada una de estas dimensiones tiene una utilidad específica en el análisis EEG:

* Separar ensayos EEG (dim 1) → Para analizar respuestas en diferentes sujetos o repeticiones.
* Filtrar por electrodos (dim 2) → Permite identificar actividad en diferentes regiones del cerebro.
* Separar bandas de frecuencia (dim 3) → Analiza cómo los ritmos cerebrales cambian en el tiempo.
* Observar evolución temporal (dim 4) → Permite analizar cambios en la actividad cerebral en diferentes momentos.
* Convertir la señal a frecuencia (dim 5) → Permite aplicar algoritmos de clasificación o análisis espectral.

**Conclusiónes**
El proceso convierte la señal EEG en una representación tiempo-frecuencia multibanda, donde:

* Cada ensayo EEG se divide en canales.
* Cada canal se filtra en 5 bandas de frecuencia (ritmos cerebrales).
* Cada banda se transforma en coeficientes tiempo-frecuencia.


In [ ]:
import matplotlib.pyplot as plt

ritmo = ['delta','theta','alpha','beta','gamma']
trial = 0
n_trials, n_canales, n_muestras, n_bands = Xrc.shape  # Simulación de datos

esp = 2 #espaciado canales
fig,ax = plt.subplots(5,1,figsize=(8,40))
# Graficar cada canal en un subplot banda respectiva
for b in range(f_bank.shape[0]): #bandas
    plot_eeg(Xrc[trial,:,:,b],tv,ax=ax[b],channels=channels,title=f'EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]}')
plt.show()

## Visualización de las señales de EEG en la frecuencia

In [ ]:
#señal orignal
Xwo = np.fft.rfft(X,axis=-1)
vfreq = np.fft.rfftfreq(X.shape[2],1/new_fs)

Xwo.shape
plt.plot(vfreq,20*np.log10(np.abs(Xwo[trial])).T)
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud [dB]')
plt.title('Eespectro Señal EEG original')
plt.show()


## Ejercicio 4

Discuta la gráfica anterior

**Solucion**

La gráfica muestra el espectro de frecuencia de varias señales EEG, mostrando cómo la energía de cada una de las señales se distribuye en diferentes frecuencias. Se observa la magnitud en decibeles (dB) vs. frecuencia en Hertz (Hz).

**1** **Descripción General de la Gráfica**
* Eje X (Frecuencia en Hz): Representa la frecuencia de la señal EEG, cubriendo un rango aproximado de 0 a 125 Hz.
* Eje Y (Magnitud en dB): Indica la intensidad de la señal en cada frecuencia.
* Múltiples colores: Pueden representar diferentes canales EEG o múltiples ensayos (trials).

**2. Características Notables en la Gráfica**
a) Predominio de Energía en Frecuencias Bajas (0-20 Hz)
Se observa un pico de alta energía en frecuencias bajas (cercanas a 0 Hz), lo que es típico en señales EEG.
La mayor parte de la actividad EEG se concentra en el rango 0-20 Hz, lo que concuerda con la presencia de ritmos cerebrales como:

* Delta (0.5-4 Hz) → Relacionado con sueño profundo.
* Theta (4-8 Hz) → Asociado con relajación y somnolencia.
* Alpha (8-13 Hz) → Presente en estados de calma y ojos cerrados.
* Beta (13-30 Hz) → Relacionado con actividad mental y concentración.

Este patrón es esperado en EEG, ya que la actividad cerebral relevante ocurre principalmente en frecuencias bajas.

**b) Presencia de Ruido en Frecuencias Altas (>30 Hz)**
A partir de 30 Hz, la magnitud disminuye, pero sigue habiendo bastante ruido.
Este rango incluye ondas Gamma (30-100 Hz), que están asociadas con procesamiento cognitivo avanzado.
Sin embargo, la dispersión de energía en frecuencias altas sugiere la presencia de ruido electromagnético o muscular.

El EEG es sensible a interferencias, y el ruido de alta frecuencia puede deberse a:

* Ruido muscular (EMG): Movimientos faciales, parpadeos.
* Interferencias de red eléctrica (50/60 Hz): Se observa un pico alrededor de 60 Hz, lo que sugiere interferencia.
* Artefactos electrónicos: Equipos en el entorno pueden introducir ruido en estas frecuencias.

**3. Mejoras Posibles en el Procesamiento de la Señal EEG**
Para mejorar la calidad de la señal y eliminar el ruido, se pueden aplicar técnicas de procesamiento:

* Filtrado Paso Bajo (<40 Hz):

Ayudaría a eliminar el ruido de alta frecuencia y aislar los ritmos cerebrales más relevantes.

* Filtrado Notch en 60 Hz:

Útil para eliminar interferencia de la red eléctrica.

* Filtrado de Bandas Específicas (IIR o FIR):

Permitiría aislar ritmos específicos (Delta, Theta, Alpha, Beta, Gamma).

* Análisis Tiempo-Frecuencia (Wavelet Transform o STFT):

Mejoraría la interpretación al permitir analizar cómo cambian las frecuencias en el tiempo.

In [ ]:
#espectro señales filtradas
Xwb = np.fft.rfft(Xrc,axis=2)

Xwb.shape

In [ ]:
#espectro señales filtradas por bandas - ritmos cerebrales

fig,ax = plt.subplots(5,1,figsize=(8,40))
# Graficar cada canal en un subplot banda respectiva
for b in range(f_bank.shape[0]): #bandas
    ax[b].plot(vfreq,20*np.log10(np.abs(Xwb[trial,:,:,b])).T)
    ax[b].set_xlabel('Frecuencia [Hz]')
    ax[b].set_ylabel('Magnitud [dB]')
    ax[b].set_title(f'Esepctro EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]}')

plt.show()

## Ejercicio 5

Discuta las gráficas

**solucion**

Las gráficas muestran del espectro de una señal EEG después de aplicar filtros en diferentes bandas de frecuencia correspondientes a los ritmos cerebrales Delta, Theta, Alpha, Beta y Gamma. vamos a analizar cada grafico basadops en la actividad cerebral.

* Eje X (Frecuencia en Hz): Muestra el rango de frecuencias analizadas.
* Eje Y (Magnitud en dB): Representa la potencia de la señal EEG en cada frecuencia.
* Múltiples colores: Representan diferentes canales EEG o  en otro caso múltiples ensayos.
* Cada gráfico ha sido filtrado en una banda específica de frecuencia, aislando un ritmo cerebral de interes para nosotros.


**Espectro EEG Filtrado en 0.5 - 4.0 Hz (Ritmo Delta)**

* Dominancia de frecuencias muy bajas (0.5 - 4 Hz), con una caída progresiva en las frecuencias más altas.
* Alta magnitud en la banda baja, lo cual es típico de ondas Delta.
* Estas ondas son características del sueño profundo y estados inconscientes.

**Espectro EEG Filtrado en 4.0 - 8.0 Hz (Ritmo Theta)**

* Se observa un pico claro en el rango de 4-8 Hz, con una disminución progresiva hacia las frecuencias más altas.
* Este ritmo está relacionado con estados de relajación, y meditacion.
* Se observa menos energía en comparación con Delta, pero aún con una presencia significativa.

**Espectro EEG Filtrado en 8.0 - 13.0 Hz (Ritmo Alpha)**

* Aparece un pico claro en la banda Alpha (8-13 Hz).
* Las ondas Alpha son más fuertes cuando una persona está relajada pero despierta, especialmente con los ojos cerrados.
* Esta actividad suele predominar en el lóbulo occipital y parietal.
  
**Espectro EEG Filtrado en 13.0 - 32.0 Hz (Ritmo Beta)**

* Se observa un pico en la banda Beta (13-32 Hz), con alta actividad en esta región.
* Las ondas Beta están relacionadas con actividad mental, atención, concentración y procesamiento sensorial.
* La caída en frecuencias altas sugiere pérdida de energía debido al filtrado.


**Espectro EEG Filtrado en 32.0 - 100.0 Hz (Ritmo Gamma)**

* Amplia presencia de actividad en el rango Gamma (32-100 Hz).
* Se observa una distribución uniforme de energia en la banda.
* Las ondas Gamma están relacionadas con procesos cognitivos avanzados, memoria de trabajo y percepción sensorial.





## Visualización de espectrogramas

Consultar qué es la Short Time Fourier Transform



In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
from scipy.signal import stft #
nperseg = 0.5*new_fs#longitud ventas en muestras
vfs,t,Xstft = stft(X,fs=new_fs,nperseg=nperseg,axis=2)
Xstft = 20*np.log10(abs(Xstft))

#graficar stft para un trial y un canal
trail = 0
chi = channels.index('C4')

fig, ax = plt.subplots(2, 1,figsize=(10,6))

ax[1].plot(tv,X[trail,chi,:])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstft[trail,chi])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Esepctrograma EEG Original -- Ch = {channels[chi]}')
print(Xstft.shape)

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 2
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Esepctrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')


# Ejercicio 6

Presente las gráficas de stft para distintos canales en los 5 ritmos cerebrales y discuta.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import stft

# Nombre de los ritmos cerebrales
ritmo = ['delta','theta','alpha','beta','gamma']

# Elegimos un trial de ejemplo
trial = 0

# Canales que queremos visualizar
channels_to_plot = ['C3', 'C4', 'O1', 'O2']  # Prueba de canales

# Ventana STFT de 0.5 segundos (puedes ajustarla a tu gusto)
nperseg = int(0.5 * new_fs)

for ch_name in channels_to_plot:
    # Índice del canal en tu lista 'channels'
    ch_idx = channels.index(ch_name)

    # Creamos una figura con 5 subplots (uno por cada banda)
    fig, axs = plt.subplots(1, 5, figsize=(20,4))

    for b in range(5):
        # Obtenemos la señal ya filtrada en la banda 'b' para el canal 'ch_idx' y trial seleccionado
        signal_band = Xrc[trial, ch_idx, :, b]

        # Calculamos la STFT
        f, t, Zxx = stft(signal_band, fs=new_fs, nperseg=nperseg)

        # Convertimos magnitud a dB (añadiendo un offset para evitar log(0))
        Zxx_db = 20 * np.log10(np.abs(Zxx) + 1e-16)

        # Graficamos el espectrograma con pcolormesh
        im = axs[b].pcolormesh(t, f, Zxx_db, shading='gouraud', cmap='jet')

        axs[b].set_title(f'Canal: {ch_name}\nRitmo: {ritmo[b]}')
        axs[b].set_xlabel('Tiempo [s]')
        axs[b].set_ylabel('Frecuencia [Hz]')

        # Barra de color en cada subplot
        plt.colorbar(im, ax=axs[b], orientation='vertical')

    plt.tight_layout()
    plt.show()

**Discucion de las graficas**
Cada señal se puede mostrar como evoluciona a travez del tiempo, por ejemplo si se observa un incremento en la potencia alpha en un momento determinado, podria estar en un estado de relaajacion.
En algunos momentos y frecuencias, podrían verse “picos” o bandas de alta energía que no correspondan a actividad cerebral sino a artefactos de parpadeo, movimiento muscular, ruido de línea (50/60 Hz), etc.
En resumen, estas gráficas STFT brindan una vista detallada de cómo la potencia de la señal EEG varía en el tiempo y en frecuencia. Esto permite:

Identificar bandas relevantes para tu estudio,por ejemplo alpha en occipital, beta en canales centrales.
Reconocer la presencia de artefactos.
Relacionar la actividad cerebral con eventos experimentales.
Observar diferencias entre canales y entre sujetos.

## Visualización de señales EEG sobre montaje 10-20

In [ ]:
import mne

# Cargar el montaje estándar
easycap_montage = mne.channels.make_standard_montage("standard_1020")


# Crear un montaje personalizado con los electrodos seleccionados
custom_pos = {ch: easycap_montage.get_positions()["ch_pos"][ch] for ch in channels}
custom_montage = mne.channels.make_dig_montage(ch_pos=custom_pos, coord_frame="head")

# Mostrar el montaje personalizado
custom_montage.plot(show_names=True)
fig = custom_montage.plot(kind="3d", show_names=True, show=False)
fig.gca().view_init(azim=70, elev=15)  # Ajustar la vista 3D

In [ ]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.visualizations.git

# Topomaps

In [ ]:
from gcpds.visualizations.topoplots import topoplot


trial = 150
vec_topo_o = abs(X[trial,:]).mean(axis=-1)
vec_topo_b = abs(Xrc[trial,:,:,:]).mean(axis=1)


fig,ax = plt.subplots(1,6,figsize=(20,10))
topoplot(vec_topo_o, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[0],show=False,vlim=(min(vec_topo_o), max(vec_topo_o)))

for b in range(f_bank.shape[0]):
    vec_ = vec_topo_b[:,b]
    topoplot(vec_, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[b+1],show=False,vlim=(min(vec_), max(vec_)))
    ax[b+1].set_title(ritmo[b])

ax[0].set_title(f'EEG-suj={sbj}-trial={trial}')

plt.show()

## Ejercicio 7

Discuta

**Discucion de las graficas**

Cada mapa topográfico representa la intensidad de la actividad EEG en cada ritmo cerebral, lo que nos permite analizar qué regiones del cerebro están más activas en un momento específico.

🔴 **1. Gráfico de EEG Original**
Muestra la actividad EEG sin filtrar, lo que significa que contiene información de todas las frecuencias.

Se observan regiones con mayor actividad en la parte occipital y temporal, lo que podría estar relacionado con procesamiento visual o motor.

🌑 **2. Ritmo Delta (0.5 - 4 Hz)**
Actividad más prominente en regiones parietales y occipitales.

Las ondas delta están asociadas con sueño profundo y estados de reposo.

La baja actividad en la parte frontal sugiere que el sujeto no estaba en un estado de sueño profundo.

🟠 3. Ritmo Theta (4 - 8 Hz)
Mayor actividad en la parte frontal y temporal.

Las ondas theta están relacionadas con relajación, memoria y procesamiento cognitivo.

La alta intensidad en la parte superior sugiere que el sujeto podría estar en un estado de concentración o relajación profunda.

🟡 4. Ritmo Alpha (8 - 13 Hz)
Se observa alta actividad en la región occipital y parietal.

Las ondas alpha son más intensas cuando una persona está relajada con los ojos cerrados.

Si el sujeto estaba en reposo, este patrón es esperado y normal.

🔴 5. Ritmo Beta (13 - 32 Hz)
Alta actividad en las regiones frontales y temporales.

Las ondas beta están relacionadas con concentración, actividad mental y procesamiento sensorial.

Esto sugiere que el sujeto podría haber estado realizando una tarea cognitiva.

🔴 6. Ritmo Gamma (32 - 100 Hz)
Actividad distribuida en múltiples regiones, pero con puntos de alta intensidad.

Las ondas gamma están asociadas con procesamiento cognitivo avanzado, memoria de trabajo y toma de decisiones.

La presencia de actividad gamma sugiere que el sujeto podría haber estado en una tarea compleja o en un estado de atención alta.

## Common Spatial Patterns

Consulté qué son los Common Spatial Patterns (CSP) y su aplicación al procesado de señales EEG

In [ ]:
import mne
from mne.decoding import CSP

# Instancia del objeto CSP
n_components = 2
csp = CSP(n_components=n_components, log= True, transform_into='average_power')
# Ajuste y transformación de los datos
csp_data = csp.fit_transform(X.astype(np.float64), y)

In [ ]:
print("CSP Transformado Shape:", csp_data.shape)
plt.scatter(csp_data[:,0],csp_data[:,1],c=y)
plt.show()

In [ ]:
#EEG original
fig,ax = plt.subplots(1,n_components,figsize=(5,5))
for cc in range(n_components):
    vec_ = np.abs(csp.filters_[cc])
    topoplot(vec_, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[cc],show=False,vlim=(min(vec_), max(vec_)))
    ax[cc].set_title(f'CSP {cc+1}')


In [ ]:
#lectura de datos
sbj = 14
X, y = load_GIGA(sbj=sbj, **load_args)

f_bank = np.array([[0.5,4.],[4., 8.],[8.,13.],[13.,32.],[32.,100.]])
vwt = np.array([[0.25, 1.75],[1.5,3],[2.75,4.25],[4,5.5],[5.25,6.75]]) #2.5 - 5 MI 0 - 7 trial completo
tf_repr = TimeFrequencyRpr(sfreq = new_fs, f_bank = f_bank,vwt=vwt)
X_ = np.squeeze(tf_repr.transform(X))
X_.shape

In [ ]:
# csp por ventanas y ritmos
# Definir las dimensiones del arreglo
ritmos_ = f_bank.shape[0]
ventanas_ = vwt.shape[0]
n_comp = 2
# Inicializar el arreglo vacío con listas anidadas
csp_M = [[None for _ in range(ventanas_)] for _ in range(ritmos_)]
csp_filters_ = np.zeros((ritmos_,ventanas_,X_.shape[1],X_.shape[1])) #ritmos ventanas Ch
Xcsp_ = np.zeros((X_.shape[0],n_comp,ritmos_,ventanas_))

for i in range(ritmos_):
    for j in range(ventanas_):
        print(f'CSP ritmo {f_bank[i]} -- ventana {vwt[j]}...')
        csp_M[i][j] =  CSP(n_components=n_comp, log= True, transform_into='average_power')
        Xcsp_[:,:,i,j] = csp.fit_transform(X_[:,:,:,j,i].astype(np.float64), y)
        csp_filters_[i,j,:] = np.abs(csp.filters_)

In [ ]:
# graficar topomaps
fig, ax = plt.subplots(ritmos_,ventanas_,figsize=(12,12))

for i in range(ritmos_):
    for j in range(ventanas_):
        vec_ = csp_filters_[i,j,0]
        vec_ = vec_/max(vec_)
        topoplot(vec_, channels, contours=3, cmap='Reds', names=None, sensors=False,ax=ax[i,j],show=False,vlim=(min(vec_), max(vec_)))
    ax[i,0].set_ylabel(ritmo[i],fontsize=20)
for j in range(ventanas_):
     ax[0,j].set_title(f'{vwt[j,0]}--{vwt[j,1]} [s]',fontsize=15)

plt.subplots_adjust(hspace=-0.025,wspace=-0.025)
plt.show()

In [ ]:
#scatters
fig, ax = plt.subplots(ritmos_,ventanas_,figsize=(12,12))

for i in range(ritmos_):
    for j in range(ventanas_):
        ax[i,j].scatter(Xcsp_[:,0,i,j],Xcsp_[:,1,i,j],c=y)
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])
    ax[i,0].set_ylabel(ritmo[i],fontsize=20)
for j in range(ventanas_):
     ax[0,j].set_title(f'{vwt[j,0]}--{vwt[j,1]} [s]',fontsize=15)

plt.subplots_adjust(hspace=0.1,wspace=0.1)
plt.show()

In [ ]:
# calculamos las matrices de coectividades (correlación)

N = X.shape[0]
connectivities_v = np.array([np.corrcoef(X[i]) for i in range(N)])

print(connectivities_v.shape)

In [ ]:
from gcpds.visualizations.connectivities import CircosConnectivity

N = len(channels)  # channels

areas = {
    'Frontal': ['Fpz', 'AFz', 'Fz', 'FCz'],
    'Frontal Right': ['Fp2','AF4','AF8','F2','F4','F6','F8',],
    'Central Right': ['FC2','FC4','FC6','FT8','C2','C4','C6','T8','CP2','CP4','CP6','TP8',],
    'Posterior Right': ['P2','P4','P6','P8','P10','PO4','PO8','O2',],
    'Central': ['Cz'],
    'Posterior': ['CPz','Pz', 'Cz','POz','Oz','Iz',],
    'Posterior Left': ['P1','P3','P5','P7','P9','PO3','PO7','O1',],
    'Central Left': ['FC1','FC3','FC5','FT7','C1','C3','C5','T7','CP1','CP3','CP5','TP7',],
    'Frontal Left': ['Fp1','AF3','AF7','F1','F3','F5','F7',],
}


conn = CircosConnectivity(
    connectivities_v[0,:,:], channels, areas=areas, threshold=0.7,

    # cmaps and themes
    areas_cmap='Set3',
    arcs_cmap='Wistia',
    hemisphere_color='lightgray',
    channel_color='#f8f9fa',
    min_alpha=0,


    # Texts
    width={'hemispheres':35, 'areas':100, 'channels':60},
    text={'hemispheres':40, 'areas':20,  'channels':40},
    separation={'hemispheres':10, 'areas':-30, 'channels':5},
    labelposition={'hemispheres':60, 'areas':0, 'channels':-10},
    size=10,
    labelsize=15,


    # Shapes
    show_emisphere=True,
    arcs_separation=30,
    connection_width=0.1,
    small_separation=5,
    big_separation=10,
    offset=0,
)

conn.figure;


## Topoplots

In [ ]:
# promediemos por canales

connecitivities_mean = np.mean(connectivities_v[0],axis=1)


## Solucion proyecto final

## 1.2

Presente una infografía que ilustre en qué consisten las señales de electroencefalografía (EEG) y su potencial aplicación en
sistemas de interaccin cerebro computador (Brain Computer Interfaces- BCI) relacionados con el paradigma de imaginaci´ on
motora (Motor Imagery- MI).

https://github.com/1004516/PROYECTO_FINAL/blob/main/Infografia.pdf



## 2.2
Con base en el estudio de la transformada Z y el diseño de filtros digitales presentado en el cuaderno Transformada Z, explique en qué consisten los filtros FIR e IIR, y como se utilizan para resaltar patrones espaciales, temporales y espectrales desde señales EEG en tareas de MI.

En el cuaderno de **Transformada Z y filtros digitales** se mostró que un SLIT discreto puede modelarse, al igual que en tiempo continuo con ecuaciones diferenciales, mediante **ecuaciones en diferencias** de la forma:

$$ \sum_{k=0}^{N} a_k y[n-k] = \sum_{k=0}^{M} b_k x[n-k] $$

Al aplicar la **transformada Z** y usar que $Z\{x[n-k]\} = z^{-k}X(z)$, dicha ecuación se convierte en una relación algebraica entre $Y(z)$ y $X(z)$:

$$ \left( \sum_{k=0}^{N} a_k z^{-k} \right) Y(z) = \left( \sum_{k=0}^{M} b_k z^{-k} \right) X(z) $$

De aquí se obtiene la **función de transferencia** del sistema:

$$ H(z) = \frac{Y(z)}{X(z)} = \frac{\sum_{k=0}^{M} b_k z^{-k}}{\sum_{k=0}^{N} a_k z^{-k}} $$

Esta expresión, escrita como cociente de polinomios en $z^{-1}$, permite clasificar los filtros digitales en dos ramas principales según los coeficientes del denominador: **filtros FIR** (cuando solo aparece $a_0$) y **filtros IIR** (cuando intervienen también $a_1, a_2, \dots, a_N$).

* En qué consisten los filtros FIR e IIR?

Tenemos la función de transferencia

$$ H(z) = \frac{\sum_{k=0}^{M} b_k z^{-k}}{\sum_{k=0}^{N} a_k z^{-k}}, $$

los filtros se entienden así:

### Filtros FIR (Finite Impulse Response)

Un filtro FIR es aquel en el que el denominador solo tiene el término $a_0$ (que normalmente se toma como 1) y los demás coeficientes $a_k = 0$ para $k \ge 1$. Entonces,

$$ H(z) = \frac{\sum_{k=0}^{M} b_k z^{-k}}{1} $$

y en el tiempo la ecuación queda:

$$ y[n] = \sum_{k=0}^{M} b_k x[n-k]. $$

En otras palabras, el filtro calcula la salida usando únicamente la muestra actual de la señal y un número limitado de muestras anteriores, sin reutilizar salidas anteriores del propio filtro. Por eso, cuando le entra un impulso, su efecto solo dura unas pocas muestras y luego desaparece, y se dice que el filtro tiene una respuesta al impulso finita.

### Filtros IIR (Infinite Impulse Response)

En un filtro IIR el denominador tiene, además de $a_0$, otros coeficientes $a_1, a_2, \dots, a_N$ distintos de cero:

$$ H(z) = \frac{\sum_{k=0}^{M} b_k z^{-k}}{\sum_{k=0}^{N} a_k z^{-k}}. $$

Al llevar esto al dominio del tiempo, la ecuación queda:

$$ a_0y[n] + a_1y[n-1] + \dots + a_Ny[n-N] = \sum_{k=0}^{M} b_k x[n-k], $$

y al despejar $y[n]$ se observa que, para calcular la salida en el instante
𝑛, el filtro utiliza tanto valores anteriores de la señal de entrada como valores anteriores de la propia salida. Como el sistema se alimenta de estos valores previos de salida, el efecto de un solo impulso puede prolongarse durante mucho tiempo (en teoría, no desaparecer por completo), y por eso a estos filtros se les conoce como de respuesta al impulso infinita (IIR).

 * como se utilizan para resaltar patrones **espaciales** desde señales EEG en tareas de MI.

 En el contexto de tareas de imaginación motora (MI), los filtros digitales FIR e IIR se utilizan sobre las señales EEG para resaltar los patrones espaciales asociados a diferentes movimientos imaginados. En primer lugar, se aplica a cada canal de EEG (por ejemplo, C3, Cz, C4, etc.) un filtro FIR o IIR diseñado como pasa–banda en las frecuencias de interés, típicamente en las bandas μ (8–13 Hz) y β (13–30 Hz), junto con filtros adicionales para atenuar el ruido y la interferencia de la red eléctrica. De esta manera, en cada electrodo se conserva principalmente la actividad relacionada con los ritmos motores y se reducen componentes no relevantes.

En segundo lugar, a partir de las señales ya filtradas, se calcula para cada canal una medida de potencia o energía en dichas bandas de frecuencia. Finalmente, estos valores se comparan entre diferentes posiciones del casco de electrodos, lo que permite identificar qué regiones muestran mayores cambios durante la MI (por ejemplo, variaciones más marcadas en C3 para la mano derecha y en C4 para la mano izquierda). Este procedimiento hace que los filtros FIR e IIR sean una etapa fundamental para limpiar la señal y realzar las diferencias entre electrodos, facilitando así la visualización y análisis de los patrones espaciales asociados a la actividad motora imaginada.


* como se utilizan para resaltar patrones **temporales** desde señales EEG en tareas de MI.

En tareas de imaginación motora (MI), los filtros digitales FIR e IIR se emplean para resaltar patrones temporales en las señales EEG, es decir, los cambios que se producen antes, durante y después del periodo en el que el sujeto imagina el movimiento. En una primera etapa, se aplican filtros pasa–altos, pasa–bajos, pasa–banda y, cuando es necesario, filtros “notch” (por ejemplo, para atenuar la interferencia de 50/60 Hz). Estos filtros, implementados como FIR o IIR, permiten eliminar tendencias lentas (deriva), ruido de alta frecuencia y componentes periódicas no relacionadas con la tarea, dejando una señal más limpia en el dominio del tiempo.

Una vez filtrado el EEG, se segmentan las señales alrededor de los eventos de interés (por ejemplo, la aparición de una pista visual o el inicio de la imaginación del movimiento) y se analizan las variaciones de amplitud y forma de onda a lo largo del tiempo. De esta manera, es posible observar con mayor claridad cómo evoluciona la actividad cerebral durante la preparación, ejecución imaginada y relajación del movimiento, identificando fenómenos como descensos o incrementos de actividad en intervalos temporales específicos (ERD/ERS). En conjunto, el uso de filtros FIR e IIR constituye una etapa clave para suprimir artefactos y realzar la estructura temporal de la señal, facilitando la interpretación de los cambios dinámicos asociados a la imaginación motora.


* como se utilizan para resaltar patrones **espectrales** desde señales EEG en tareas de MI.

En tareas de imaginación motora (MI), los filtros digitales FIR e IIR se utilizan para resaltar patrones espectrales en las señales EEG, es decir, los cambios en la distribución de energía de la señal a lo largo de las distintas frecuencias. En particular, se busca aislar las bandas relacionadas con la actividad motora, como los ritmos μ (aprox. 8–13 Hz) y β (aprox. 13–30 Hz). Para ello, se diseñan filtros pasa–banda FIR o IIR centrados en estas bandas de frecuencia y se aplican de forma independiente a cada canal de EEG. De esta manera, se atenúan las componentes que no pertenecen a dichas bandas y se enfatiza la actividad espectral asociada a la MI.

Una vez filtrada la señal en las bandas de interés, se calcula la potencia espectral o la energía de cada canal en ventanas de tiempo definidas (por ejemplo, antes, durante y después del periodo de imaginación del movimiento). Esto permite observar con mayor claridad fenómenos como disminuciones de potencia (ERD) o aumentos de potencia (ERS) en las bandas μ y β, que están directamente relacionados con la preparación y ejecución imaginada del movimiento. En conjunto, el uso de filtros FIR e IIR como filtros pasa–banda constituye una herramienta fundamental para aislar las frecuencias relevantes y realzar los patrones espectrales característicos de la imaginación motora en EEG.

## Punto 2.3
Adaptar y modificar el código base del cuaderno del punto 2.1 para mejorar la detección de patrones de imaginación motora en distintos sujetos y condiciones experimentales. Discuta las dificultades encontradas respecto a lo estudiado en el estado del arte, especialmente, respecto a la inconsistencia en la identificación de patrones espaciales, temporales y frecuenciales en estas señales entre sujetos y estímulos.
<!--  -->
# SOLUCION

En este trabajo se adaptó y modificó el código base del cuaderno del punto 2.1 con el objetivo de mejorar la detección de patrones de imaginación motora en distintos sujetos y condiciones experimentales. Para ello, se implementó la carga por lotes de varios participantes, se aplicó una normalización por sujeto sobre los ensayos de EEG y se utilizó una representación tiempo–frecuencia basada en un banco de filtros y ventanas de tiempo. Sobre esta representación se entrenó un esquema de FBCSP (Filter-Bank CSP) combinado con validación cruzada agrupada por sujeto, de forma que los patrones extraídos se evalúan dejando sujetos completos fuera del entrenamiento.

In [ ]:
# Carga de varios sujetos y evaluación de CSP con banco de filtros (filter-bank)
from mne.decoding import CSP
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Sacamos la lista de sujetos desde la base de datos.
# Si no existe ese atributo, usamos por defecto los sujetos del 1 al 5.
sbj_list = getattr(db, "subjects", None)
if sbj_list is None:
    sbj_list = list(range(1, 6))
print(f"Using {len(sbj_list)} subjects: {sbj_list}")

# Definimos las bandas de frecuencia donde suele aparecer la imaginación motora
# Cada fila de f_bank es [frecuencia_inicial, frecuencia_final] en Hz
f_bank = np.array([[4.0, 8.0], [8.0, 13.0], [13.0, 26.0], [26.0, 35.0]])

# Ventanas de tiempo (en segundos) donde vamos a mirar la señal respecto al estímulo
# Cada fila es [tiempo_inicial, tiempo_final]
vwt = np.array([[0.5, 2.5], [2.0, 4.0], [3.5, 5.5]])  # ventanas de 2.0 s

# Objeto que se encarga de aplicar el banco de filtros y las ventanas de tiempo
tf_repr = TimeFrequencyRpr(sfreq=new_fs, f_bank=f_bank, vwt=vwt)


def center_scale_trials(X: np.ndarray) -> np.ndarray:
    """Subject-wise centering and scaling to reduce between-subject amplitude drift."""
    # Restamos la media en el tiempo para que cada canal quede centrado en 0
    X = X - X.mean(axis=2, keepdims=True)
    # Dividimos por la desviación estándar para que la escala de amplitud sea parecida
    # entre ensayos (evitamos división por 0 sumando un valor muy pequeño)
    X = X / (X.std(axis=2, keepdims=True) + 1e-6)
    return X


def load_subject_tf(sbj: int):
    # Cargamos los datos crudos del sujeto con la función load_GIGA
    X, y = load_GIGA(sbj=sbj, **load_args)
    # Normalizamos los ensayos de este sujeto
    X = center_scale_trials(X)
    # Aplicamos la representación tiempo-frecuencia y quitamos ejes extra
    return np.squeeze(tf_repr.transform(X)), y


# Aquí vamos a juntar los datos de todos los sujetos
# X_all: señales, y_all: etiquetas, groups: a qué sujeto pertenece cada ensayo
X_all, y_all, groups = [], [], []
for sbj in sbj_list:
    # Obtenemos los datos tiempo-frecuencia de este sujeto
    X_tf, y = load_subject_tf(sbj)
    # Guardamos los datos y las etiquetas en listas
    X_all.append(X_tf)
    y_all.append(y)
    # En groups repetimos el número de sujeto tantas veces como ensayos tenga
    groups.extend([sbj] * len(y))

# Unimos todos los sujetos en un solo arreglo grande
X_all = np.concatenate(X_all, axis=0)
y_all = np.concatenate(y_all, axis=0)
groups = np.asarray(groups)
print("Time-frequency tensor:", X_all.shape, "labels:", y_all.shape)


class FBCSPFeatures(BaseEstimator, TransformerMixin):
    """Filter-bank CSP over bands and windows with log-variance features."""

    def __init__(self, n_components: int = 2):
        # Cantidad de componentes CSP que vamos a sacar por cada banda y ventana
        self.n_components = n_components

    def fit(self, X, y):
        # X tiene forma (ensayos, canales, tiempo, ventanas, bandas)
        # Aquí entrenamos un CSP para cada combinación banda–ventana
        self.csp_bank_ = []
        n_bands = X.shape[-1]     # cuántas bandas de frecuencia hay
        n_windows = X.shape[-2]   # cuántas ventanas de tiempo hay

        for b in range(n_bands):
            for w in range(n_windows):
                # Creamos un CSP para esta banda y esta ventana
                csp = CSP(
                    n_components=self.n_components,
                    log=True,
                    transform_into="average_power",
                )
                # Entrenamos el CSP usando solo esa banda y ventana
                csp.fit(X[:, :, :, w, b], y)
                # Guardamos la pareja (banda, ventana, modelo CSP)
                self.csp_bank_.append((b, w, csp))
        return self

    def transform(self, X):
        # Aplicamos todos los CSP entrenados para sacar las características
        feats = []
        for b, w, csp in self.csp_bank_:
            # Extraemos las características CSP de esa banda y ventana
            feats.append(csp.transform(X[:, :, :, w, b]))
        # Unimos todas las características en una sola matriz
        return np.concatenate(feats, axis=1)


# Definimos en cuántas partes queremos dividir los datos para validación cruzada
# Usamos entre 2 y 5 particiones, dependiendo de cuántos sujetos haya
n_splits = max(2, min(5, len(np.unique(groups))))

# GroupKFold se asegura de que los datos de un mismo sujeto estén juntos,
# es decir, un sujeto no se parte entre entrenamiento y prueba
gkf = GroupKFold(n_splits=n_splits)

# Creamos un pipeline que:
# 1) Saca características con FBCSP
# 2) Estandariza las características
# 3) Aplica una regresión logística para clasificar
clf = Pipeline(
    steps=[
        ("fbcsp", FBCSPFeatures(n_components=2)),
        ("scaler", StandardScaler()),
        (
            "logreg",
            LogisticRegression(
                max_iter=1000,
                class_weight="balanced",  # ayuda cuando hay más ejemplos de una clase que de otra
                solver="lbfgs",
            ),
        ),
    ]
)

# Evaluamos el modelo usando validación cruzada por sujeto.
# scoring="roc_auc" mide qué tan bien separa las clases (área bajo la curva ROC).
scores = cross_val_score(
    clf, X_all, y_all, groups=groups, cv=gkf, scoring="roc_auc"
)
print("Group-wise (subject-held-out) AUC:", scores, "mean=", scores.mean())


# Dificultades vs estado del arte

A partir de estas modificaciones se observaron varias dificultades que coinciden con lo reportado en el estado del arte, especialmente respecto a la inconsistencia de los patrones espaciales, temporales y frecuenciales entre sujetos y estímulos:

**Inconsistencia espacial:**
Las fuentes sensorimotores no aparecen exactamente en los mismos electrodos para todos los sujetos (diferencias en anatomía, colocación del casco, etc.). Por eso, los filtros CSP ajustados en un sujeto no se transfieren bien a otro sin algún tipo de alineamiento. En la literatura se mencionan enfoques como alineamiento Riemanniano o estimación de fuentes para hacer más estable la estructura de covarianzas entre sujetos.

**Variabilidad espectral:**
Las bandas μ y β no son fijas; sus picos pueden desplazarse por fatiga, medicamentos o cambios en la impedancia de los electrodos. Aunque aquí se usan bandas fijas con un banco de filtros, esto puede no capturar bien los picos específicos de cada persona. En el estado del arte se proponen enfoques más adaptativos, como selección de picos por sujeto o modelos espectrales personalizados.

**Desfase temporal (temporal jitter):**
El inicio de la imaginación motora y la dinámica ERD/ERS no ocurre exactamente al mismo tiempo en todos los ensayos. Al trabajar con ventanas temporales fijas, se corre el riesgo de dejar por fuera parte de la información relevante. Por eso, en trabajos recientes se usan ventanas deslizantes más flexibles o métodos que ajustan la latencia de forma automática, aunque suelen requerir más datos y modelos más complejos.

**Ruido no estacionario:**
Artefactos musculares, oculares y ruido de línea cambian entre sesiones y sujetos, afectando tanto la covarianza como el espectro de la señal. En este código solo se aborda parcialmente el problema con una normalización por ensayo/sujeto, mientras que en el estado del arte se utilizan técnicas más avanzadas como ICA, SSP o estimadores de covarianza más robustos.

**Generalización entre condiciones:**
La imaginación motora (por ejemplo, izquierda/derecha) puede verse mezclada con movimientos reales, diferencias en las instrucciones o cambios en las condiciones del experimento. En la literatura se proponen métodos de adaptación de dominio y meta–aprendizaje para manejar estos cambios. En nuestro caso, el uso de FBCSP con banco de filtros, varias ventanas de tiempo y validación cruzada agrupada por sujeto ayuda a reducir parte de la variabilidad, pero las inconsistencias espaciales, temporales y espectrales siguen siendo un cuello de botella importante, tal como se discute en los trabajos actuales sobre MI–BCI.


# 2.4

Presente un nuevo cuaderno con la solución propuesta, indicando la influencia de artefactos fisiológicos (ej., parpadeo, actividad muscular) y factores genéticos, cognitivos y de experiencia previa, relacionados con la variabilidad de las señales entre sujetos.

Aquí se retoma la solución implementada en el punto 2.3 (banco de filtros, ventanas de tiempo, FBCSP y validación cruzada agrupada por sujeto) y se analiza cómo distintos factores del sujeto y de la señal influyen en la variabilidad entre participantes y, en consecuencia, en el rendimiento del modelo, teniendo en cuenta que las señales de EEG pueden cambiar bastante de una persona a otra por varios motivos.

Primero, **los artefactos fisiológicos** afectan mucho la señal. Por ejemplo, los parpadeos y los movimientos de los ojos generan picos grandes en los electrodos frontales, y la actividad muscular en cara y cuello mete energía en frecuencias altas (más de 30–40 Hz). Si no se controlan, estos artefactos pueden “engañar” a CSP y hacer que aprenda más ruido que imaginación motora. Por eso en la solución se considera el uso de filtrado (notch, pasaaltos) y la revisión de ensayos muy ruidosos que podrían ser descartados.

También influyen **factores fisiológicos y genéticos**. Cada sujeto tiene diferencias en su cerebro y en cómo se organizan las áreas motoras, así que las bandas μ y β no aparecen exactamente igual para todos. Esto ayuda a explicar por qué, aunque usemos el mismo banco de filtros, algunos sujetos muestran patrones más claros que otros.

A esto se suman **factores cognitivos**, como la fatiga o el nivel de atención. Cuando una persona está cansada o distraída, la potencia de las bandas μ/β y la profundidad del ERD cambian, y eso se nota en el desempeño del modelo. La normalización por sujeto y el uso de varias ventanas temporales ayudan un poco a compensar estos cambios de estado.

Por último, **la experiencia previa y la estrategia de imaginación** también juegan un papel. No todos imaginan el movimiento de la misma forma (algunos se enfocan más en “sentir” el movimiento y otros en visualizarlo), y no todos tienen la misma práctica. Esto hace que la activación aparezca en lugares y tiempos diferentes. Con las múltiples ventanas de tiempo y la validación dejando sujetos por fuera se puede ver que el modelo tiene que lidiar con toda esta variabilidad, y que por eso no siempre generaliza perfecto entre personas.

## 2.5

Presente una infografía que resuma los conceptos y desarrollos implementados respecto al reconocimiento de señales de EEG en
tareas de MI.

https://github.com/david-1192/Senales_y_Sistemas_2025/blob/main/EEG%20se%C3%B1ales%20info.pdf